In [1]:
import pandas as pd
import numpy as np
import itertools
import lightgbm as lgbm
import matplotlib.pyplot as plt

from script import *
from feature_engineering import *
import db_operations as dbop
from constants import *
import data_process as dp

import time

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
pd.set_option('display.max_rows',200)
idx = pd.IndexSlice
agg_operations =['mean','median','max','min','std','size']

In [3]:
start = 20100101
cursor = dbop.connect_db("sqlite3").cursor()

df_d = dbop.create_df(cursor, STOCK_DAY[TABLE], start=start)
df_d = dp.proc_stock_d(dp.prepare_stock_d(df_d))
df_d.drop(columns=['open0','high0','low0','vol0'],inplace=True)
df_d['pct'] = df_d.sort_index().groupby('code')['close'].pct_change()*100
df_d['amt']/=1e5
df_d = df_d.astype('float32')
print(df_d.columns)
print(df_d.shape)
df_d.info(memory_usage='deep')
df_d.tail()

df_d_basic = dbop.create_df(cursor, STOCK_DAILY_BASIC[TABLE], start=start)
df_d_basic = dp.prepare_stock_d_basic(df_d_basic)
#
df_d_basic["pb*pe_ttm"] = df_d_basic["pb"] * df_d_basic["pe_ttm"]
df_d_basic["pb*pe"] = df_d_basic["pb"] * df_d_basic["pe"]

df_d_basic = df_d_basic.astype('float32')
print(df_d_basic.columns)
print(df_d_basic.shape)
df_d_basic.info(memory_usage='deep')
# df_d_basic.index.tail()

df = df_d
periods = [5,10,20,30,60,120,250]
for k in periods:
    df['{:d}ma'.format(k)]=df.reset_index('code').groupby('code')['close'].rolling(k).mean()
print(df.columns)
    
for col2 in ["{}ma".format(k) for k in periods]:
    col1 = 'close'
    df['{}/{}'.format(col1,col2)] = df[col1]/df[col2]
print(df.columns)

select * from stock_day where date>=20100101
Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct'],
      dtype='object')
(6951987, 11)
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 6951987 entries, (000001.SZ, 2010-01-04 00:00:00) to (688399.SH, 2020-04-21 00:00:00)
Data columns (total 11 columns):
open          float32
high          float32
low           float32
close         float32
vol           float32
amt           float32
adj_factor    float32
avg           float32
close0        float32
avg0          float32
pct           float32
dtypes: float32(11)
memory usage: 638.7 MB


open        high         low       close  \
code      date                                                         
688399.SH 2020-04-15   80.750000   84.339996   80.300003   82.000000   
          2020-04-16   83.400002   96.000000   82.440002   94.900002   
          2020-04-17   95.949997   95.949997   88.879997   92.000000   
          2020-04-20   95.790001  109.970001   95.199997  108.730003   
          2020-04-21  108.000000  118.199997  102.000000  114.440002   

                               vol       amt  adj_factor         avg  \
code      date                                                         
688399.SH 2020-04-15  10474.320312  0.863751         1.0   82.463715   
          2020-04-16  31260.830078  2.815096         1.0   90.051849   
          2020-04-17  24047.009766  2.208113         1.0   91.824860   
          2020-04-20  34636.898438  3.590274         1.0  103.654602   
          2020-04-21  33081.828125  3.580782         1.0  108.240166   

                          close0        avg0        pct  
code      date                                           
688399.SH 2020-04-15   82.000000   82.463715   2.641129  
          2020-04-16   94.900002   90.051849  15.731705  
          2020-04-17   92.000000   91.824860  -3.055853  
          2020-04-20  108.730003  103.654602  18.184780  
          2020-04-21  114.440002  108.240166   5.251539

select * from stock_daily_basic where date>=20100101
Index(['close', 'turnover_rate', 'turnover_rate_f', 'volume_ratio', 'pe',
       'pe_ttm', 'pb', 'ps', 'ps_ttm', 'total_share', 'float_share',
       'free_share', 'total_mv', 'circ_mv', 'pb*pe_ttm', 'pb*pe'],
      dtype='object')
(6573157, 16)
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 6573157 entries, (000001.SZ, 2010-01-04 00:00:00) to (688399.SH, 2020-04-21 00:00:00)
Data columns (total 16 columns):
close              float32
turnover_rate      float32
turnover_rate_f    float32
volume_ratio       float32
pe                 float32
pe_ttm             float32
pb                 float32
ps                 float32
ps_ttm             float32
total_share        float32
float_share        float32
free_share         float32
total_mv           float32
circ_mv            float32
pb*pe_ttm          float32
pb*pe              float32
dtypes: float32(16)
memory usage: 426.5 MB
Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj

In [4]:
df = df.join(df_d_basic.drop(columns='close'),how='inner')

In [5]:
index = df_d.index[(df_d['close/120ma']<=0.69)
                 & (df_d['close/30ma']<=0.778)
                 & (df_d['close/60ma']<=0.65)
                 & (df_d['close/60ma']>=0.55)
                ]
# print(df['turnover_rate_f'].head())
# df['p10mean_turnover_rate_f'] = df.reset_index('code').groupby('code')['turnover_rate_f'].rolling(10).mean()
df['p10mean_turnover_rate'] = df.reset_index('code').groupby('code')['turnover_rate'].rolling(10).mean()

stock_mask = (df['circ_mv']>50e4) & (df['total_mv']>100e4) & (df['close0']>2.5) & (df['p10mean_turnover_rate']<5) & (df['amt']>0.5)
print(stock_mask.sum(),stock_mask.mean())
# print(len(index))

1252056 0.19053506152470234


In [6]:
start_date = '-'.join([str(start)[:4],str(start)[4:6],str(start)[6:8]])
df_r_spl = pd.read_parquet(r"database\return_spl_5%_10%_20_8_15%").loc[idx[:,start_date:],:].sort_index()

In [7]:
features = []
for col2 in ["{}ma".format(k) for k in periods]:
    col1 = 'close'
    features.append('{}/{}'.format(col1,col2))
print(features)
# result_50 = ml.assess_feature3(df.loc[stock_mask,features],df_r.r,q_bin=50)
# result_100 = ml.assess_feature3(df.loc[stock_mask,features],df_r.r,q_bin=100)

['close/5ma', 'close/10ma', 'close/20ma', 'close/30ma', 'close/60ma', 'close/120ma', 'close/250ma']


In [43]:
print(result.sort_values(['median_std','mean_std'],ascending=False))
print(result_50.sort_values(['median_std','mean_std'],ascending=False))
print(result_100.sort_values(['median_std','mean_std'],ascending=False))


             mean_std  median_std  mean_q96%  median_q96%  mean_q4%  \
close/60ma   0.002214    0.002930   0.006737    -0.002907  0.000177   
close/30ma   0.001786    0.002642   0.005013    -0.004687 -0.000075   
close/20ma   0.001518    0.002552   0.004747    -0.005438  0.000216   
close/120ma  0.002525    0.002524   0.006875    -0.003664  0.000394   
close/10ma   0.001263    0.002277   0.004025    -0.005446  0.000436   
close/5ma    0.001634    0.002189   0.005813    -0.005129  0.000456   
close/250ma  0.001411    0.001058   0.005713    -0.006659  0.001670   

             median_q4%  
close/60ma    -0.011214  
close/30ma    -0.011815  
close/20ma    -0.011823  
close/120ma   -0.010571  
close/10ma    -0.011761  
close/5ma     -0.011686  
close/250ma   -0.009766  
             mean_std  median_std  mean_q96%  median_q96%  mean_q4%  \
close/60ma   0.001779    0.002629   0.003023    -0.005295 -0.001769   
close/30ma   0.001455    0.002506   0.001915    -0.006860 -0.001968   
close/120m

In [8]:
# strategy_mask = (
#     (df['close/120ma']<0.7) 
#     & (df['close/60ma']<0.75) & (df['close/60ma']>0.55) 
#     & (df['close/30ma']<0.79) 
#     & (df['close/20ma']<0.82)
#     & (df['close/5ma']<0.95)
#                 )
strategy_mask = (
    (df['close/120ma']<0.72) 
    & (df['close/60ma']<0.77) 
    & (df['close/60ma']>0.55) 
    & (df['close/30ma']<0.82) 
#     & (df['close/20ma']<0.82)
#     & (df['close/5ma']<0.95)
                )
print(strategy_mask.sum())

21819


In [9]:
df_r_spl.loc[df.index[stock_mask & strategy_mask],'r'].agg(agg_operations)
# print(df_r_spl2.loc[df.index[stock_mask & strategy_mask],'r'].agg(agg_operations))
df_r_spl.loc[df.index[stock_mask & strategy_mask],'r'].reset_index('code')['r'].resample('Q').agg(agg_operations)
df_r_spl.loc[df.index[stock_mask & strategy_mask],:].sort_values('date',ascending=False)[:50]
# df[stock_mask & strategy_mask][['close']].sort_values('date',ascending=False)[:50]

# df.loc[idx['300207.SZ',:],df_d.columns].sort_values('date',ascending=False)[:20]

mean         0.028712
median       0.024468
max          0.944298
min         -0.554711
std          0.087808
size      3201.000000
Name: r, dtype: float64

,mean,median,max,min,std,size
date,,,,,,
2011-12-31,-0.010115,-0.002491,0.081329,-0.103537,0.053515,24
2012-03-31,0.122152,0.121635,0.149573,0.098592,0.016839,7
2012-06-30,NaN,NaN,NaN,NaN,NaN,0
2012-09-30,NaN,NaN,NaN,NaN,NaN,0
2012-12-31,0.049252,0.053061,0.076647,0.019380,0.020764,5
2013-03-31,NaN,NaN,NaN,NaN,NaN,0
2013-06-30,-0.000289,-0.000463,0.094919,-0.057143,0.029956,60
2013-09-30,0.027094,0.028235,0.110083,-0.066289,0.052430,37
2013-12-31,-0.014565,-0.029872,0.055400,-0.062069,0.040722,9


buy_at         max  max_idx   idx  is_selled  \
code      date                                                           
002506.SZ 2020-04-21         NaN         NaN      NaN  1881      False   
          2020-04-20   14.118690   14.628389   1881.0  1880      False   
300207.SZ 2020-04-13   91.053711  102.341362   2132.0  2130       True   
002411.SZ 2020-04-10   35.994240   38.868778   2095.0  2092       True   
002583.SZ 2020-04-03   33.788223   36.149460   2103.0  2100       True   
002411.SZ 2020-04-03   37.077396   38.910439   2089.0  2088       True   
002583.SZ 2020-04-02   34.013103   36.149460   2103.0  2099       True   
002411.SZ 2020-04-02   32.994720   38.910439   2089.0  2087       True   
601808.SH 2020-04-01   12.939040   14.795610   2487.0  2485       True   
002411.SZ 2020-04-01   32.161518   36.369179   2088.0  2086       True   
300618.SZ 2020-04-01  109.310043  121.387108    752.0   748       True   
002583.SZ 2020-04-01   32.832481   36.149460   2103.0  2098       True   
002353.SZ 2020-04-01  183.529388  207.722992   2348.0  2346       True   
002123.SZ 2020-04-01   97.448639  146.759995   2357.0  2351       True   
300618.SZ 2020-03-31  106.319305  121.387108    752.0   747       True   
002411.SZ 2020-03-31   32.328159   36.369179   2088.0  2085       True   
002123.SZ 2020-03-31   97.448639  102.658615   2352.0  2350       True   
300766.SZ 2020-03-31   26.150000   28.969999    252.0   248       True   
002353.SZ 2020-03-31  178.804703  203.568542   2347.0  2345       True   
002583.SZ 2020-03-31   32.495163   36.149460   2103.0  2097       True   
300766.SZ 2020-03-30   26.000000   28.969999    252.0   247       True   
601808.SH 2020-03-30   12.802361   13.531321   2484.0  2483       True   
002123.SZ 2020-03-30  101.484535  101.998192   2350.0  2349       True   
002411.SZ 2020-03-30   34.452820   34.452820   2085.0  2084       True   
300618.SZ 2020-03-30  106.547607  112.506233    748.0   746       True   
002583.SZ 2020-03-30   34.181763   34.350422   2097.0  2096       True   
002353.SZ 2020-03-30  178.397400  203.568542   2347.0  2344       True   
300618.SZ 2020-03-27  106.091011  112.506233    748.0   745       True   
002411.SZ 2020-03-27   34.577801   35.994240   2084.0  2083       True   
002353.SZ 2020-03-27  178.071564  203.568542   2347.0  2343       True   
          2020-03-26  192.734360  193.630417   2343.0  2342       True   
002411.SZ 2020-03-26   33.744602   36.494160   2083.0  2082       True   
002506.SZ 2020-03-26   17.839500   18.094351   1865.0  1864       True   
300618.SZ 2020-03-26  112.985672  113.236794    745.0   744       True   
002411.SZ 2020-03-25   38.285538   39.493675   2082.0  2081       True   
002506.SZ 2020-03-25   18.145321   20.184120   1864.0  1863       True   
002353.SZ 2020-03-25  192.734360  194.445023   2342.0  2341       True   
          2020-03-24  192.978745  200.391602   2341.0  2340       True   
002411.SZ 2020-03-24   40.243557   41.118420   2081.0  2080       True   
300618.SZ 2020-03-24  111.821335  115.451309    743.0   742       True   
002008.SZ 2020-03-23  356.953125  378.648529   2459.0  2457       True   
300618.SZ 2020-03-23  108.807777  115.451309    743.0   741       True   
002411.SZ 2020-03-23   40.951778   41.160080   2080.0  2079       True   
300748.SZ 2020-03-23   27.178278   30.822800    363.0   360       True   
002563.SZ 2020-03-23   32.643448   36.692249   2183.0  2179       True   
002387.SZ 2020-03-23   39.458221   41.379543   2268.0  2265       True   
002353.SZ 2020-03-23  192.897278  200.391602   2341.0  2339       True   
002411.SZ 2020-03-20   41.326717   42.451538   2079.0  2078       True   
          2020-03-19   42.076599   42.826477   2078.0  2077       True   
000963.SZ 2020-03-19  203.533234  250.912689   2447.0  2433       True   

                         sell_at  sell_date         r  
code      date                                         
002506.SZ 2020-04-21         NaN        NaT       NaN  
         

In [106]:
df.loc[strategy_mask,['open','close','close0','amt','pct','adj_factor','close/30ma','close/60ma','close/120ma']].sort_values('date',ascending=False)[:50]
# stock_mask.sort_index(level='date').tail(50)
# print(stock_mask.loc[idx['300207.SZ',:],:].tail())
print(df.loc[idx['300207.SZ',:],['circ_mv','total_mv','close0','p10mean_turnover_rate_f','amt','turnover_rate_f','turnover_rate']].tail())

                         circ_mv    total_mv  close0  p10mean_turnover_rate_f  \
code      date                                                                  
300207.SZ 2020-04-07  2178567.25  2466680.75   15.72                  4.87429   
          2020-04-08  2197969.25  2488648.75   15.86                  4.93659   
          2020-04-09  2156393.50  2441574.50   15.56                  4.85817   
          2020-04-10  1981775.50  2243863.50   14.30                  5.13992   
          2020-04-13  1815472.75  2055567.25   13.10                  5.69921   

                            amt  turnover_rate_f  turnover_rate  
code      date                                                   
300207.SZ 2020-04-07   9.109506           6.2898         4.2001  
          2020-04-08   6.978767           4.7744         3.1882  
          2020-04-09   7.080351           4.9246         3.2885  
          2020-04-10   9.848499           7.2678         4.8532  
          2020-04-13  11.169043     

In [56]:
df_r[df_r['r'].isnull()].is_selled.unique()

array([False])

In [10]:
index = df.index[(df['close/120ma']<=0.69)
                 & (df['close/30ma']<=0.778)
                 & (df['close/60ma']<=0.65)
                 & (df['close/60ma']>=0.55)
                ]
print(len(index))

3705


In [13]:

df.loc[index,features].sort_values('date',ascending=False)[:50]
# .sort_values(['code','date'])
# df.loc[idx['002506.SZ',:],features].sort_values('date',ascending=False)[:20]
df_r_spl.loc[index,:].sort_values('date',ascending=False)[:50]
df_r_spl.loc[index,'r'].agg(agg_operations)
df_r_spl.loc[index,:].reset_index('code')['r'].resample('Q').agg(agg_operations)


close/5ma  close/10ma  close/20ma  close/30ma  \
code      date                                                        
002506.SZ 2020-04-21   0.986563    0.974162    0.892086    0.732540   
          2020-04-20   0.972708    0.961272    0.860015    0.714225   
          2020-04-17   1.001410    0.975275    0.848141    0.713269   
          2020-04-16   0.999296    0.967632    0.820691    0.697903   
          2020-04-15   1.005567    0.976351    0.808166    0.695269   
          2020-04-14   1.013793    0.988568    0.795455    0.692580   
          2020-04-10   0.960509    0.933637    0.725664    0.647709   
          2020-04-09   0.990753    0.950269    0.737010    0.662057   
          2020-04-08   0.991464    0.928374    0.723353    0.653397   
          2020-04-07   1.007218    0.905338    0.717122    0.651113   
          2020-04-03   0.964401    0.834033    0.678429    0.620532   
          2020-04-02   0.971519    0.811096    0.679730    0.628583   
002411.SZ 2020-04-02   0.992004    0.901760    0.772298    0.707119   
          2020-04-01   0.949414    0.862662    0.738137    0.682737   
002506.SZ 2020-04-01   0.940353    0.775213    0.666451    0.622398   
          2020-03-31   0.875723    0.722805    0.637224    0.600634   
002411.SZ 2020-03-31   0.898301    0.833064    0.716108    0.668707   
300618.SZ 2020-03-31   0.969373    0.949786    0.832150    0.765647   
002353.SZ 2020-03-31   0.973751    0.941733    0.809090    0.717764   
002411.SZ 2020-03-30   0.914879    0.855889    0.740975    0.698408   
300618.SZ 2020-03-30   0.958847    0.933060    0.816704    0.745695   
002353.SZ 2020-03-30   0.940909    0.903983    0.768080    0.686508   
002506.SZ 2020-03-30   0.878415    0.742417    0.672062    0.639418   
002411.SZ 2020-03-27   0.907912    0.851495    0.743001    0.706852   
002506.SZ 2020-03-27   0.821006    0.719533    0.666000    0.639728   
002353.SZ 2020-03-27   0.981820    0.938401    0.792599    0.710567   
002411.SZ 2020-03-26   0.920258    0.870336    0.766098    0.734931   
300618.SZ 2020-03-23   0.918125    0.834413    0.762911    0.680974   
600083.SH 2020-03-13   0.939013    0.893529    0.691910    0.578481   
          2020-03-12   0.942584    0.911084    0.681523    0.588829   
          2020-03-11   0.957528    0.929442    0.679504    0.603966   
          2020-03-10   0.971763    0.932075    0.667458    0.610608   
          2020-03-09   0.943550    0.893694    0.632256    0.593108   
          2020-03-06   0.994289    0.911348    0.647530    0.620838   
          2020-03-05   1.011121    0.883519    0.636837    0.622756   
          2020-03-04   0.966839    0.817602    0.604307    0.599022   
          2020-03-03   0.970934    0.797658    0.615891    0.616360   
          2020-03-02   0.890056    0.707717    0.576944    0.581769   
          2020-02-28   0.844277    0.661041    0.575859    0.586329   
600892.SH 2020-02-06   0.974359    0.824096    0.683863    0.649532   
          2020-02-05   0.909091    0.756781    0.649008    0.622228   
603012.SH 2020-02-04   0.858841    0.808333    0.776310    0.754165   
603226.SH 2020-02-04   0.850650    0.788688    0.753884    0.736946   
300013.SZ 2020-02-04   0.851254    0.775763    0.684635    0.647727   
600892.SH 2020-02-04   0.835489    0.693133    0.615942    0.597632   
600069.SH 2020-02-04   0.831712    0.733906    0.681139    0.665542   
000662.SZ 2020-02-04   0.843080    0.752010    0.682311    0.645603   
002634.SZ 2020-02-04   0.877679    0.822226    0.766191    0.734082   
002205.SZ 2020-02-04   0.865250    0.813293    0.786608    0.768930   
002634.SZ 2020-02-03   0.880902    0.846740    0.792142    0.763864   

                      close/60ma  close/120ma  close/250ma  
code      date                                              
002506.SZ 2020-04-21    0.595327     0.515117     0.469021  
          2020-04-20    0.587241     0.510295     0.465849  
          2020-04-17    0.593873     0.518295     0.474386  
          2020-04-16    

buy_at         max  max_idx   idx  is_selled  \
code      date                                                           
002506.SZ 2020-04-21         NaN         NaN      NaN  1881      False   
          2020-04-20   14.118690   14.628389   1881.0  1880      False   
          2020-04-17   14.322570   14.628389   1881.0  1879      False   
          2020-04-16   14.628389   15.087121   1879.0  1878       True   
          2020-04-15   14.526450   15.087121   1879.0  1877       True   
          2020-04-14   15.138090   15.392941   1877.0  1876       True   
          2020-04-10   14.322570   14.985181   1876.0  1874       True   
          2020-04-09   15.392941   15.392941   1874.0  1873       True   
          2020-04-08   15.494880   15.494880   1873.0  1872       True   
          2020-04-07   15.545850   15.596820   1872.0  1871       True   
          2020-04-03   15.494880   15.749730   1871.0  1870       True   
          2020-04-02   15.494880   15.851670   1870.0  1869       True   
002411.SZ 2020-04-02   32.994720   38.910439   2089.0  2087       True   
          2020-04-01   32.161518   36.369179   2088.0  2086       True   
002506.SZ 2020-04-01   15.545850   15.851670   1870.0  1868       True   
          2020-03-31   15.392941   16.157492   1868.0  1867       True   
002411.SZ 2020-03-31   32.328159   36.369179   2088.0  2085       True   
300618.SZ 2020-03-31  106.319305  121.387108    752.0   747       True   
002353.SZ 2020-03-31  178.804703  203.568542   2347.0  2345       True   
002411.SZ 2020-03-30   34.452820   34.452820   2085.0  2084       True   
300618.SZ 2020-03-30  106.547607  112.506233    748.0   746       True   
002353.SZ 2020-03-30  178.397400  203.568542   2347.0  2344       True   
002506.SZ 2020-03-30   16.412340   16.718161   1867.0  1866       True   
002411.SZ 2020-03-27   34.577801   35.994240   2084.0  2083       True   
002506.SZ 2020-03-27   16.973009   17.482710   1866.0  1865       True   
002353.SZ 2020-03-27  178.071564  203.568542   2347.0  2343       True   
002411.SZ 2020-03-26   33.744602   36.494160   2083.0  2082       True   
300618.SZ 2020-03-23  108.807777  115.451309    743.0   741       True   
600083.SH 2020-03-13   11.770000   12.110000   2340.0  2339       True   
          2020-03-12   11.310000   12.110000   2340.0  2338       True   
          2020-03-11   12.350000   12.360000   2338.0  2337       True   
          2020-03-10   12.990000   13.100000   2337.0  2336       True   
          2020-03-09   12.490000   13.100000   2337.0  2335       True   
          2020-03-06   13.300000   13.380000   2335.0  2334       True   
          2020-03-05   13.550000   13.970000   2334.0  2333       True   
          2020-03-04   13.670000   14.000000   2333.0  2332       True   
          2020-03-03   13.790000   14.000000   2333.0  2331       True   
          2020-03-02   13.440000   14.160000   2331.0  2330       True   
          2020-02-28   13.000000   14.160000   2331.0  2329       True   
600892.SH 2020-02-06   22.130501   26.177219   2285.0  2283       True   
          2020-02-05   20.612980   23.774479   2284.0  2282       True   
603012.SH 2020-02-04   13.858401   15.651840   1188.0  1182       True   
603226.SH 2020-02-04   24.718979   28.516541    652.0   639       True   
300013.SZ 2020-02-04   21.951542   25.034882   2273.0  2271       True   
600892.SH 2020-02-04   19.032230   21.687891   2283.0  2281       True   
600069.SH 2020-02-04   10.777200   11.931900   2318.0  2317       True   
000662.SZ 2020-02-04   20.908781   23.809422   2098.0  2095       True   
002634.SZ 2020-02-04   33.686401   38.419201   1983.0  1970       True   
002205.SZ 2020-02-04   14.765520   17.370430   2431.0  2421       True   
002634.SZ 2020-02-03   31.320000   35.495998   1971.0  1969       True   

                         sell_at  sell_date         r  
code      date                                         
002506.SZ 2020-04-21         NaN        NaT       NaN  
         

mean         0.059319
median       0.055263
max          1.204987
min         -0.735319
std          0.119687
size      3705.000000
Name: r, dtype: float64

,mean,median,max,min,std,size
date,,,,,,
2011-09-30,0.076663,0.076663,0.076663,0.076663,NaN,1
2011-12-31,-0.027651,-0.030303,0.124654,-0.157175,0.062038,53
2012-03-31,0.073331,0.068966,0.436782,-0.099422,0.089724,59
2012-06-30,NaN,NaN,NaN,NaN,NaN,0
2012-09-30,NaN,NaN,NaN,NaN,NaN,0
2012-12-31,0.049924,0.094584,0.146096,-0.104598,0.100050,9
2013-03-31,NaN,NaN,NaN,NaN,NaN,0
2013-06-30,NaN,NaN,NaN,NaN,NaN,0
2013-09-30,NaN,NaN,NaN,NaN,NaN,0


In [67]:
df_r[(df_r.is_selled==True) & (df_r.r.isnull())].sort_values('date').head(50)

,,buy_at,max,max_idx,idx,is_selled,sell_at,r
code,date,,,,,,,
000418.SZ,2019-04-30,167.895859,175.670792,2205.0,2203,True,NaN,NaN
002450.SZ,2019-07-03,52.103348,53.277512,1912.0,1910,True,NaN,NaN
600401.SH,2019-07-04,2.830200,3.207560,1392.0,1391,True,NaN,NaN
002477.SZ,2019-10-11,3.269920,3.883030,2016.0,2014,True,NaN,NaN
002143.SZ,2019-11-26,0.895160,0.927130,1984.0,1983,True,NaN,NaN
600240.SH,2020-01-21,3.030120,3.349080,2335.0,2333,True,NaN,NaN
000848.SZ,2020-03-12,145.541504,161.439117,2488.0,2471,True,NaN,NaN
600236.SH,2020-03-12,30.058800,31.209702,2409.0,2395,True,NaN,NaN
600729.SH,2020-03-12,81.136620,87.456543,2317.0,2305,True,NaN,NaN


In [74]:
df_r['curr_date']= df_r.index.get_level_values('date')

In [75]:
df_r.head()

buy_at         max  max_idx  idx  is_selled  \
code      date                                                          
000001.SZ 2010-01-04  852.767456  858.153320      1.0    0       True   
          2010-01-05  834.814453  850.254028      5.0    1       True   
          2010-01-06  822.247375  850.254028      5.0    2       True   
          2010-01-07  807.884949  850.254028      5.0    3       True   
          2010-01-08  843.790955  850.254028      5.0    4       True   

                         sell_date     sell_at         r  curr_date  
code      date                                                       
000001.SZ 2010-01-04  1.262909e+18  807.884949 -0.052632 2010-01-04  
          2010-01-05  1.263254e+18  810.398376 -0.029247 2010-01-05  
          2010-01-06  1.263254e+18  810.398376 -0.014411 2010-01-06  
          2010-01-07  1.263254e+18  810.398376  0.003111 2010-01-07  
          2010-01-08  1.263341e+18  785.264221 -0.069362 2010-01-08

In [78]:
df_r1 = pd.read_parquet(r"database\return_5%_10%_20_8_15% close").loc[idx[:,start_date:],:].sort_index()

C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:707: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:734: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [79]:
df_r2 = pd.read_parquet(r"database\return_5%_10%_20_8_15%").loc[idx[:,start_date:],:].sort_index()

C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:707: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:734: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [81]:
print(df_r.r.agg(agg_operations)[:4])
print(df_r1.r.agg(agg_operations)[:4])
print(df_r2.r.agg(agg_operations)[:4])
print(df_r1.r.agg(agg_operations)==df_r2.r.agg(agg_operations))

mean       0.002731
median    -0.009649
max       12.109637
min       -0.872727
Name: r, dtype: float64
mean       0.003762
median    -0.015353
max       12.109637
min       -0.872727
Name: r, dtype: float64
mean       0.003762
median    -0.015353
max       12.109637
min       -0.872727
Name: r, dtype: float64
mean      True
median    True
max       True
min       True
std       True
size      True
Name: r, dtype: bool


In [84]:
print(df_r.loc[index,'r'].reset_index('code')['r'].resample('Q').agg(agg_operations))
print(df_r1.loc[index,'r'].reset_index('code')['r'].resample('Q').agg(agg_operations))

                mean    median       max       min       std  size
date                                                              
2011-09-30  0.076663  0.076663  0.076663  0.076663       NaN     1
2011-12-31 -0.035069 -0.034591  0.081654 -0.134314  0.072813    13
2012-03-31  0.113472  0.092634  0.436782 -0.046875  0.112979    18
2012-06-30       NaN       NaN       NaN       NaN       NaN     0
2012-09-30       NaN       NaN       NaN       NaN       NaN     0
2012-12-31  0.136266  0.136266  0.146096  0.126437  0.013901     2
2013-03-31       NaN       NaN       NaN       NaN       NaN     0
2013-06-30       NaN       NaN       NaN       NaN       NaN     0
2013-09-30       NaN       NaN       NaN       NaN       NaN     0
2013-12-31       NaN       NaN       NaN       NaN       NaN     0
2014-03-31       NaN       NaN       NaN       NaN       NaN     0
2014-06-30       NaN       NaN       NaN       NaN       NaN     0
2014-09-30       NaN       NaN       NaN       NaN       NaN  

In [85]:
print(df_r.loc[index,'r'].agg(agg_operations))
print(df_r1.loc[index,'r'].agg(agg_operations))

mean         0.064221
median       0.058891
max          1.204987
min         -0.554711
std          0.126345
size      1302.000000
Name: r, dtype: float64
mean         0.073225
median       0.048387
max          1.204987
min         -0.554711
std          0.145928
size      1302.000000
Name: r, dtype: float64


In [122]:
print(df.index.is_lexsorted())
# df.groupby('code').apply(lambda x:np.arange(1,len(x+1))).head()
df['tmp'] = 1
df['idx'] = df.reset_index('code').groupby('code')['tmp'].expanding(1).sum().astype('int')
print(df['idx'].head())
del df['tmp']

KeyError: 'tmp'

In [ ]:
# df_target_not_trash = df[stock_mask & strategy_mask]
print(df_target_not_trash.shape)
for i in range(1,6):
#     df_target_not_trash['idx_diff{}'.format(i)] = df_target_not_trash.groupby('code')['idx'].diff(i)
    # 打印连续超跌i天的样本数量
    print('idx_diff{0}=={0}:'.format(i),(df_target_not_trash['idx_diff{}'.format(i)]==i).sum())
    
# df_target_not_trash = df_target_not_trash.join(df_r,how='inner',rsuffix='r_')
df_target_not_trash = df_target_not_trash.join(df_r2[['r']],how='inner',rsuffix='2')

# 检查拼表后数据是否完整
print(df_target_not_trash.shape)
print('间隔超跌的样本数：',(df_target_not_trash['idx_diff1']>1).sum())
print(df_target_not_trash[df_target_not_trash['idx_diff1']>1]['idx_diff1'].head())
print('第一次超跌的样本数：',df_target_not_trash['idx_diff1'].isnull().sum())
print(df_target_not_trash[df_target_not_trash['idx_diff1'].isnull()]['idx_diff1'].head())

In [128]:
# df_target = df.loc[index]
# print(df.columns)
# df_target = df_target.join(df_r,how='inner',rsuffix='_r')
# df_target = df_target.join(df_r2,how='inner',rsuffix='_r2')
# print(df_target.shape)
for i in range(1,6):
#     df_target['idx_diff{}'.format(i)] = df_target.groupby('code')['idx'].diff(i)
    # 打印连续超跌i天的样本数量
    print('idx_diff{0}=={0}:'.format(i),(df_target['idx_diff{}'.format(i)]==i).sum())
    print(df_target.loc[df_target['idx_diff{}'.format(i)]==i,'r'].agg(agg_operations),'\n')
    print(df_target.loc[df_target['idx_diff{}'.format(i)]==i,'r_r2'].agg(agg_operations),'\n')

for k in [1,5,10,20]:
    print('k={}'.format(k))
    print(df_target.loc[df_target['idx_diff1']>k,'r'].agg(agg_operations),'\n')
    print(df_target.loc[df_target['idx_diff1']>k,'r_r2'].agg(agg_operations),'\n')

print('\nnull')
print(df_target.loc[df_target['idx_diff1'].isnull(),'r'].agg(agg_operations),'\n')
print(df_target.loc[df_target['idx_diff1'].isnull(),'r_r2'].agg(agg_operations),'\n')

print('\nAll')
print(df_target.loc[:,'r'].agg(agg_operations),'\n')
print(df_target.loc[:,'r_r2'].agg(agg_operations),'\n')
    
# df_target = df_target.join(df_r,how='inner',rsuffix='r_')
# 检查拼表后数据是否完整
print(df_target.shape)
print('间隔超跌的样本数：',(df_target['idx_diff1']>1).sum())
print(df_target[df_target['idx_diff1']>1]['idx_diff1'].head())
print('第一次超跌的样本数：',df_target['idx_diff1'].isnull().sum())
print(df_target[df_target['idx_diff1'].isnull()]['idx_diff1'].head())

idx_diff1==1: 592
mean        0.050100
median      0.040513
max         0.655405
min        -0.554711
std         0.112235
size      592.000000
Name: r, dtype: float64 

mean        0.054878
median      0.024129
max         0.655405
min        -0.554711
std         0.132836
size      592.000000
Name: r_r2, dtype: float64 

idx_diff2==2: 323
mean        0.030315
median      0.022945
max         0.551485
min        -0.554711
std         0.096181
size      323.000000
Name: r, dtype: float64 

mean        0.032791
median      0.005263
max         0.551485
min        -0.554711
std         0.121943
size      323.000000
Name: r_r2, dtype: float64 

idx_diff3==3: 194
mean        0.012319
median      0.013333
max         0.253676
min        -0.554711
std         0.093841
size      194.000000
Name: r, dtype: float64 

mean        0.010770
median     -0.002691
max         0.253676
min        -0.554711
std         0.113661
size      194.000000
Name: r_r2, dtype: float64 

idx_diff4==4: 131
mean   

In [129]:
for i in range(1,6):
    print('idx_diff{0}=={0}:'.format(i),(df_target['idx_diff{}'.format(i)]==i).sum())
    print(df_target_not_trash.loc[df_target_not_trash['idx_diff{}'.format(i)]==i,'r'].agg(agg_operations),'\n')
    print(df_target_not_trash.loc[df_target_not_trash['idx_diff{}'.format(i)]==i,'r2'].agg(agg_operations),'\n')
    
for k in [1,5,10,20]:
    print('k={}'.format(k))
    print(df_target_not_trash.loc[(df_target_not_trash['idx_diff1']>k),'r'].agg(agg_operations),'\n')
    print(df_target_not_trash.loc[(df_target_not_trash['idx_diff1']>k),'r2'].agg(agg_operations),'\n')

print('\nNull')
print(df_target_not_trash.loc[(df_target_not_trash['idx_diff1'].isnull()),'r'].agg(agg_operations),'\n')
print(df_target_not_trash.loc[(df_target_not_trash['idx_diff1'].isnull()),'r2'].agg(agg_operations),'\n')

print('\nAll')
print(df_target_not_trash['r'].agg(agg_operations),'\n')
print(df_target_not_trash['r2'].agg(agg_operations),'\n')
# print(df_target_not_trash.loc[df_target_not_trash['idx_diff1']!=1,'r2'].agg(agg_operations),'\n')
# print(df_target_not_trash.loc[(df_target_not_trash['idx_diff1']>5)|(df_target_not_trash['idx_diff1'].isnull()),'r2'].agg(agg_operations),'\n')

df_target_not_trash['1+r'] = df_target_not_trash['r']+1
df_target_not_trash['1+r2'] = df_target_not_trash['r2']+1

df_target_not_trash = df_target_not_trash.sort_values(['date','code'])
print(df_target_not_trash['1+r'][::-1].cumprod(axis=0).tail())
print(df_target_not_trash['1+r2'][::-1].cumprod(axis=0).tail())
print()
print(df_target_not_trash['1+r'][::-1].cumprod(axis=0)[500:510])
print(df_target_not_trash['1+r2'][::-1].cumprod(axis=0)[500:510])
print()
print(df_target_not_trash['1+r'][::-1].cumprod(axis=0)[200:210])
print(df_target_not_trash['1+r2'][::-1].cumprod(axis=0)[200:210],'\n')

for k in [10,20,50,100,200]:
    print('\nk={}'.format(k))
    print(df_target_not_trash['1+r'][::-1].cumprod(axis=0)[k:k+10])
    print(df_target_not_trash['1+r2'][::-1].cumprod(axis=0)[k:k+10])


print(df_target_not_trash['1+r'][::-1].cumprod(axis=0)[:30])
print(df_target_not_trash[['1+r','1+r2','sell_date']][::-1][:30])

print(df_target_not_trash['r'].reset_index('code')['r'].resample('Q').agg(agg_operations))


mean         0.026368
median       0.022989
max          0.655405
min         -0.207692
std          0.069571
size      1728.000000
Name: r, dtype: float64 

mean         0.034785
median       0.015994
max          0.655405
min         -0.492201
std          0.093027
size      1728.000000
Name: r2, dtype: float64 

idx_diff1==1: 592
mean         0.026680
median       0.022405
max          0.268148
min         -0.171271
std          0.061703
size      1064.000000
Name: r, dtype: float64 

mean         0.036855
median       0.015085
max          0.358641
min         -0.492201
std          0.091062
size      1064.000000
Name: r2, dtype: float64 

idx_diff2==2: 323
mean        0.025853
median      0.020886
max         0.242424
min        -0.171271
std         0.061038
size      721.000000
Name: r, dtype: float64 

mean        0.035423
median      0.011073
max         0.358641
min        -0.492201
std         0.091625
size      721.000000
Name: r2, dtype: float64 

idx_diff3==3: 194
mean   

                mean    median       max       min       std  size
date                                                              
2011-12-31 -0.010072 -0.002491  0.081329 -0.103537  0.057601    20
2012-03-31  0.129921  0.123876  0.149573  0.120000  0.012359     5
2012-06-30       NaN       NaN       NaN       NaN       NaN     0
2012-09-30       NaN       NaN       NaN       NaN       NaN     0
2012-12-31  0.049252  0.053061  0.076647  0.019380  0.020764     5
2013-03-31       NaN       NaN       NaN       NaN       NaN     0
2013-06-30 -0.001443 -0.001713  0.094919 -0.057143  0.030376    54
2013-09-30  0.023118  0.026345  0.109537 -0.066289  0.052144    34
2013-12-31  0.053341  0.053341  0.055400  0.051282  0.002912     2
2014-03-31  0.051143  0.062480  0.074101  0.005512  0.031619     4
2014-06-30  0.035240  0.035240  0.078145 -0.007664  0.060676     2
2014-09-30       NaN       NaN       NaN       NaN       NaN     0
2014-12-31       NaN       NaN       NaN       NaN       NaN  

In [22]:
k = 5
ss = df.head(2000)

In [23]:
# ss[features].reset_index('code').groupby('code')[features].rolling(250*k).apply(lambda x:pd.DataFrame(x).rank(pct=True)).tail(10)
# ss[features].groupby('code').rolling(1000).apply(lambda x:pd.Series(x).shape).tail(10)
# ss[features].groupby('code').expanding(1000).apply(lambda x:pd.DataFrame(x).rank(pct=True)).tail(10)
ss[features].groupby('code')[features].rolling(100).apply(lambda x:pd.DataFrame(x).rank(pct=True).loc[len(x)-1,:]).tail(20)

C:\Users\dell-pc\Anaconda3.6\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  after removing the cwd from sys.path.


close/5ma  close/10ma  close/20ma  close/30ma  \
code      code      date                                                        
000001.SZ 000001.SZ 2018-06-12       0.44        0.49        0.39        0.44   
                    2018-06-13       0.33        0.34        0.32        0.37   
                    2018-06-14       0.61        0.59        0.51        0.51   
                    2018-06-15       0.82        0.76        0.67        0.60   
                    2018-06-19       0.34        0.33        0.39        0.35   
                    2018-06-20       0.47        0.43        0.47        0.43   
                    2018-06-21       0.40        0.38        0.46        0.41   
                    2018-06-22       0.49        0.45        0.51        0.47   
                    2018-06-25       0.12        0.15        0.20        0.14   
                    2018-06-26       0.11        0.12        0.18        0.14   
                    2018-06-27       0.07        0.07        0.06        0.05   
                    2018-06-28       0.08        0.05        0.06        0.04   
                    2018-06-29       0.44        0.16        0.12        0.12   
                    2018-07-02       0.08        0.05        0.04        0.03   
                    2018-07-03       0.21        0.10        0.07        0.04   
                    2018-07-04       0.27        0.12        0.08        0.05   
                    2018-07-05       0.41        0.22        0.10        0.07   
                    2018-07-06       0.70        0.33        0.13        0.11   
                    2018-07-09       0.96        0.93        0.50        0.39   
                    2018-07-10       0.93        0.92        0.51        0.40   

                                close/60ma  close/120ma  close/250ma  
code      code      date                                              
000001.SZ 000001.SZ 2018-06-12        0.45         0.07         0.02  
                    2018-06-13        0.33         0.02         0.01  
                    2018-06-14        0.54         0.10         0.04  
                    2018-06-15        0.62         0.16         0.08  
                    2018-06-19        0.33         0.02         0.01  
                    2018-06-20        0.46         0.04         0.02  
                    2018-06-21        0.43         0.04         0.01  
                    2018-06-22        0.45         0.05         0.01  
                    2018-06-25        0.13         0.01         0.01  
                    2018-06-26        0.12         0.01         0.01  
                    2018-06-27        0.06         0.01         0.01  
                    2018-06-28        0.02         0.01         0.01  
                    2018-06-29        0.09         0.03         0.03  
                    2018-07-02        0.01         0.01         0.01  
                    2018-07-03        0.02         0.02         0.02  
                    2018-07-04        0.02         0.02         0.02  
                    2018-07-05        0.03         0.03         0.01  
                    2018-07-06        0.05         0.05         0.04  
                    2018-07-09        0.19         0.09         0.07  
                    2018-07-10        0.19         0.09         0.07

In [33]:
for i in range(5,6):
    w = i*250
    cols = ['p{}_q1%_{}'.format(w,f) for f in features]
    print(cols)
#     print(len(cols))
#     print(len(features))
    df[cols] = df.reset_index('code').groupby('code')[features].rolling(w,min_periods=int(w*0.9)).quantile(0.01)

['p1250_q1%_close/5ma', 'p1250_q1%_close/10ma', 'p1250_q1%_close/20ma', 'p1250_q1%_close/30ma', 'p1250_q1%_close/60ma', 'p1250_q1%_close/120ma', 'p1250_q1%_close/250ma']


In [67]:
mask60 =  (df['close/30ma']<=df['p1250_q1%_close/30ma']*1.03) 
print(mask60.sum(),df['p1250_q1%_close/60ma'].notnull().sum(),mask60.sum()/df['p1250_q1%_close/60ma'].notnull().sum())

df_r.loc[df.index[mask60],'r'].agg(agg_operations)

4891 277147 0.01764767433888875


C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pandas\core\indexing.py:910: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_nested_tuple(tup)


mean         0.017229
median       0.000000
max          0.707305
min         -0.554711
std          0.099349
size      4891.000000
Name: r, dtype: float64

close  turnover_rate  turnover_rate_f  volume_ratio  \
code      date                                                              
603997.SH 2020-04-07  12.01         9.5215        39.344398          2.26   
          2020-04-08  10.81         6.7385        27.844700          1.13   
          2020-04-09   9.73         2.4960        10.313700          0.36   
          2020-04-10   9.01         8.2926        34.266300          1.14   
          2020-04-13   9.08         6.4077        26.477800          0.86   

                             pe     pe_ttm      pb      ps  ps_ttm  \
code      date                                                       
603997.SH 2020-04-07  41.295300  41.295300  2.7027  0.6829  0.6829   
          2020-04-08  37.169300  37.169300  2.4326  0.6147  0.6147   
          2020-04-09  33.455799  33.455799  2.1896  0.5533  0.5533   
          2020-04-10  30.980101  30.980101  2.0276  0.5123  0.5123   
          2020-04-13  31.220800  31.220800  2.0433  0.5163  0.5163   

                        total_share   float_share    free_share      total_mv  \
code      date                                                                  
603997.SH 2020-04-07  102360.289062  63235.328125  15303.180664  1.229347e+06   
          2020-04-08  102360.289062  63235.328125  15303.180664  1.106515e+06   
          2020-04-09  102360.289062  63235.328125  15303.180664  9.959656e+05   
          2020-04-10  102360.289062  63235.328125  15303.180664  9.222662e+05   
          2020-04-13  102360.289062  63235.328125  15303.180664  9.294314e+05   

                          circ_mv   pb*pe_ttm       pb*pe  
code      date                                             
603997.SH 2020-04-07  759456.3125  111.608803  111.608803  
          2020-04-08  683573.9375   90.418037   90.418037  
          2020-04-09  615279.7500   73.254814   73.254814  
          2020-04-10  569750.3125   62.815254   62.815254  
          2020-04-13  574176.8125   63.793457   63.793457

In [83]:
df.index.is_lexsorted()
df['p1mv_pct'] = df['pct'].groupby('code').shift(1)
df['p2mv_pct'] = df['pct'].groupby('code').shift(2)
sdb_mask = (
            ((df['close/60ma']<0.8 )) & (df['close/60ma']>0)
    & (df['close/250ma']>0.65 ) 
#             & (df['pct']<-5 ) & (df['pct']<df['p1mv_pct']) & (df['pct']<df['p2mv_pct'])
            & ((df['p1mv_pct']+df['p2mv_pct']+df['pct'])<-5)
           )

stock_mask2 = (df['circ_mv']>50e4) & (df['total_mv']>100e4) & (df['close0']>2.5) & (df['p10mean_turnover_rate']<5) & (df['amt']>0.5)
print('stock_mask2 pct:',stock_mask2.sum()/len(stock_mask2))

sdb_index = df.index[sdb_mask 
                     & stock_mask2
                    ]
sdb_index.shape

df_r_spl['r'].agg(agg_operations[:4])
df_r_spl.loc[sdb_index,'r'].agg(agg_operations)
df_r_spl.loc[sdb_index,:].reset_index('code')['r'].resample('Q').agg(agg_operations)

True

stock_mask2 pct: 0.19053506152470234


(4256,)

mean       0.003282
median    -0.009585
max       12.799429
min       -0.972953
Name: r, dtype: float64

mean         0.015181
median       0.014199
max          0.944298
min         -0.402637
std          0.109157
size      4256.000000
Name: r, dtype: float64

,mean,median,max,min,std,size
date,,,,,,
2013-06-30,0.002117,0.002197,0.094954,-0.121174,0.043520,54
2013-09-30,0.035073,0.033068,0.186016,-0.043970,0.058704,20
2013-12-31,-0.011431,-0.025633,0.055400,-0.071238,0.053787,6
2014-03-31,0.029473,0.038100,0.102305,-0.056107,0.046551,20
2014-06-30,0.007624,0.004050,0.118501,-0.083985,0.058555,23
2014-09-30,0.018899,0.018899,0.028481,0.009317,0.013551,2
2014-12-31,0.056253,0.056253,0.066475,0.046032,0.014455,2
2015-03-31,0.056662,0.055422,0.101639,0.012926,0.044370,3
2015-06-30,-0.019674,0.001301,0.233766,-0.233108,0.091477,96


KeyboardInterrupt: 

In [89]:
df.columns

Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', '5ma', '10ma', '20ma', '30ma', '60ma', '120ma',
       '250ma', 'close/5ma', 'close/10ma', 'close/20ma', 'close/30ma',
       'close/60ma', 'close/120ma', 'close/250ma'],
      dtype='object')